In [1]:
import os
# os.environ['CUDA_VISIBLE_DEVICES'] = '-1' # to disable GPU
import featureEng as fe
import processStocks as ps
import getStocks
import visualize
import predicts
import pandas as pd
import TS_LSTM as ts_lstm

pd.options.display.max_columns = 999
pd.options.display.max_rows = 999

/usr/local/anaconda/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# view tenslow enabled devices
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 10340256716394967568
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 7557644288
locality {
  bus_id: 1
  links {
  }
}
incarnation: 9299678990376011098
physical_device_desc: "device: 0, name: GeForce GTX 1070, pci bus id: 0000:65:00.0, compute capability: 6.1"
]


# User parameters

In [3]:
ticker = 'amd'.upper()
n_in = 100 # number of historical days to input
n_out = 1 # days into future to predict
target = 'd1close'

# Load Data
### Several methods available here:
* Load a single stock
* get a refresh of the stock
* load a single stock as the target while using many other stocks as additional features

In [4]:
# load dataset
# dataset = getStocks.get_single(ticker=ticker, save=True)
dataset = getStocks.load_single(ticker)
#dataset = getStocks.join_tgt_spt(target_ticker=ticker.upper(), number_spt=20)
dataset.reset_index(level=0, inplace=True)
dataset.rename(columns=lambda x: x.replace(' ',''),inplace=True)

Loaded AMD


# Prepare Data for LSTM

In [5]:
# using default options
'''
ts_data=ts_lstm.ts_data.default_prep(n_in=n_in, n_out=n_out, 
                      ticker=ticker,target=target,
                      rawData=dataset)
'''

ts_data = ts_lstm.ts_data(n_in=n_in, n_out=n_out, 
                      ticker=ticker,target=target,
                      rawData=dataset)
ts_data.eng_features(derivate=True, weekdays=False)
ts_data.roll_data()
ts_data.tscv(train=0.99)

/mnt/data/Dropbox/Projects/LSTM-forecast/src/TS_LSTM.py:90: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.data.drop('Date', axis=1, inplace=True)


# Build LSTM Architecture

In [6]:
ts_model = ts_lstm.build_model(ts_data, 
                               inlayer=int(ts_data.train_X.shape[-1])*2,
                               hiddenlayers=0,
                               loss_function='mean_squared_error',
                               dropout=0.05,
                               activation='tanh')

Compilation Time :  0.0153656005859375
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 22)                2992      
_________________________________________________________________
dropout_1 (Dropout)          (None, 22)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 23        
Total params: 3,015
Trainable params: 3,015
Non-trainable params: 0
_________________________________________________________________


# Fit model

In [ ]:
history = ts_model.fit(ts_data.train_X, ts_data.train_y, 
                    epochs=50, 
                    batch_size=1024, 
                    validation_data=(ts_data.test_X, ts_data.test_y), 
                    verbose=2, 
                    shuffle=False)

In [ ]:
visualize.plot_loss(history)

# Results

In [ ]:
# select last 30 days of test data (30 records)
test_X = ts_data.test_X[-30:]
test_Y = ts_data.test_y[-30:]

In [ ]:
test_Y[:,0]
# [print(x.shape) for x in test_X]

In [ ]:
yhat = [float(ts_model.predict(x.reshape(1, 100, 11))) for x in test_X]

In [ ]:
import matplotlib.pyplot as plt
x = [x for x in range(len(yhat))]
plt.plot(x, yhat, label='predicted change in price')
plt.plot(x, test_Y[:,0], label='actual change in price')
plt.title(ticker)
plt.xlabel('days')
plt.ylabel('amount change in price')
plt.legend()
plt.show()